# Exploration of HGDP data


The aim of this script is  to identify populations that are the most distinct by their ancestry. On the basis of this and sample quality, we will choose roughly 50-100 samples to store on the cloud as cram files.

In [1]:
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import turbo
from pprint import pprint
from gnomad.utils.annotations import annotate_adj
from hail.plot import show
from bokeh.io.export import get_screenshot_as_png
from bokeh.plotting import figure
from bokeh.io import export_png

In [2]:
import hail as hl
hl.init()
hl.plot.output_notebook()

Running on Apache Spark version 2.4.5
SparkUI available at http://my-cluster-m.australia-southeast1-a.c.katalina-dev.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/hail/hail-20210317-0559-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

The dataset we'll be using is the gnomAD HGDP + 1KG callset, which is available as a Hail matrix table from [here](https://gnomad.broadinstitute.org/downloads#v3-hgdp-1kg). 

In [3]:
mt = hl.read_matrix_table("gs://gcp-public-data--gnomad/release/3.1/mt/genomes/gnomad.genomes.v3.1.hgdp_1kg_subset_dense.mt")

Let's first explore the Hail matrix table by seeing how many rows (variants) and columns (samples) we have. 

In [4]:
mt.count_rows()

175312130

In [5]:
mt.count_cols()

3942

We're only interested in plotting the HGDP data, so now let's filter out all 1KG samples.

In [6]:
hgdp = mt.filter_cols(mt.subsets.hgdp)

In [7]:
print('After filtering, %d/3942 samples remain.' % hgdp.count_cols())

After filtering, 819/3942 samples remain.


## Sample QC

Now that we filtered for only HGDP samples, we can now perform basic QC on the data. We want to ensure that the data we're using are of the highest quality. Let's filter out any samples with low call rates and a mean depth below 30.

In [8]:
hgdp_qc = hl.sample_qc(hgdp)

In [13]:
p = hl.plot.histogram(hgdp_qc.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

We can see that a high proportion of samples have a low call rate (less than 0.9). In the gnomAD QC pipeline, a hard filter is set at a call rate of less than 0.895. Let's go ahead and filter these out.

In [14]:
hgdp_qc = hgdp_qc.filter_cols(hgdp_qc.sample_qc.call_rate >= 0.895)

Each time the sample_qc function is called, it takes a long time to run (around 10-15 minutes with 100 VMs). In order to make this process quicker, we'll keep the samples that we've already identified that have a call rate over 0.895 by filtering out all of the poor-quality samples from the original hgdp matrix table.

First, get the sample names from the hgdp_qc matrix table (our samples to keep).

In [59]:
samples_to_keep = hgdp_qc.s

In [18]:
samples_to_keep = list(set(samples_to_keep.collect()))

Let's also check how many samples we filtered out during the qc step:

In [21]:
len(samples_to_keep)

730

In total, we have 730 samples with a call rate greater than or equal to 0.895.

Now let's filter out the low-quality samples from the original matrix table.

In [83]:
set_to_keep = hl.literal(samples_to_keep)

In [87]:
hgdp = hgdp.filter_cols(~set_to_keep.contains(hgdp['s']), keep=False)

In [88]:
hgdp.count_cols()

730

From the filtered data, let's also plot another QC metric - median sample coverage.

In [90]:
p_2 = hl.plot.histogram(hgdp.bam_metrics.median_coverage, legend='Median Sample Coverage')
show(p_2)

We want to filter out samples that have a median depth between 30-40, so let's go ahead and filter those out.

In [94]:
hgdp = hgdp.filter_cols((hgdp.bam_metrics.median_coverage >= 30) & (hgdp.bam_metrics.median_coverage <= 40))

In [96]:
hgdp.count_cols()

597

After QC filtering, we have a total of 597 samples. Let's save the table.

In [111]:
filtered_table = hgdp.cols()

In [125]:
filtered_table.export('gs://katalina/filtered_table')

2021-03-17 07:53:46 Hail: INFO: Coerced sorted dataset
2021-03-17 07:53:48 Hail: INFO: merging 16 files totalling 818.3K...
2021-03-17 07:53:48 Hail: INFO: while writing:
    gs://katalina/filtered_table
  merge time: 212.302ms


## Making a PCA from the sample information

Let's assign the information we need to conduct a PCA to variables. Later, we'll be using labels and pca score information, and I found there's an error if the tables are not from the same object. To bypass this, we can assign hgdp_qc.cols to a variable, then divide this between pca scores and label names. 

In [97]:
columns = hgdp.cols()
pca_scores = columns.population_inference.pca_scores
labels = columns.labeled_subpop

2021-03-17 07:33:00 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


For colours, we can use the categorical colour mapper from Bokeh. To do this, we need to get our populations as a string. 

In [98]:
pops = list(set(labels.collect()))

In [99]:
mapper = CategoricalColorMapper(palette=turbo(47), factors=pops)

In [100]:
p1 = hl.plot.scatter(pca_scores[0], pca_scores[1], label=labels,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True, colors = mapper)
show(p1)

2021-03-17 07:33:17 Hail: INFO: Coerced sorted dataset


In [101]:
p2 = hl.plot.scatter(pca_scores[1], pca_scores[2], label=labels,
                    title='PCA', xlabel='PC2', ylabel='PC3', collect_all=True, colors = mapper)
show(p2)

2021-03-17 07:33:32 Hail: INFO: Coerced sorted dataset


In [102]:
p3 = hl.plot.scatter(pca_scores[2], pca_scores[3], label=labels,
                    title='PCA', xlabel='PC3', ylabel='PC4', collect_all=True, colors = mapper)
show(p3)

2021-03-17 07:33:40 Hail: INFO: Coerced sorted dataset


In [103]:
p4 = hl.plot.scatter(pca_scores[3], pca_scores[4], label=labels,
                    title='PCA', xlabel='PC4', ylabel='PC5', collect_all=True, colors = mapper)
show(p4)

2021-03-17 07:33:46 Hail: INFO: Coerced sorted dataset


In [104]:
p5 = hl.plot.scatter(pca_scores[4], pca_scores[5], label=labels,
                    title='PCA', xlabel='PC5', ylabel='PC6', collect_all=True, colors = mapper)
show(p5)

2021-03-17 07:33:52 Hail: INFO: Coerced sorted dataset


In [105]:
p6 = hl.plot.scatter(pca_scores[5], pca_scores[6], label=labels,
                    title='PCA', xlabel='PC6', ylabel='PC7', collect_all=True, colors = mapper)
show(p6)

2021-03-17 07:33:58 Hail: INFO: Coerced sorted dataset


In [106]:
p7 = hl.plot.scatter(pca_scores[6], pca_scores[7], label=labels,
                    title='PCA', xlabel='PC7', ylabel='PC8', collect_all=True, colors = mapper)
show(p7)

2021-03-17 07:34:04 Hail: INFO: Coerced sorted dataset


In [107]:
p8 = hl.plot.scatter(pca_scores[7], pca_scores[8], label=labels,
                    title='PCA', xlabel='PC8', ylabel='PC9', collect_all=True, colors = mapper)
show(p8)

2021-03-17 07:34:09 Hail: INFO: Coerced sorted dataset


In [108]:
p9 = hl.plot.scatter(pca_scores[8], pca_scores[9], label=labels,
                    title='PCA', xlabel='PC9', ylabel='PC10', collect_all=True, colors = mapper)
show(p9)

2021-03-17 07:34:14 Hail: INFO: Coerced sorted dataset


In [109]:
p10 = hl.plot.scatter(pca_scores[9], pca_scores[10], label=labels,
                    title='PCA', xlabel='PC10', ylabel='PC11', collect_all=True, colors = mapper)
show(p10)

2021-03-17 07:34:19 Hail: INFO: Coerced sorted dataset
